<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
Push HuggingFace
</div>

## Push to Hugging Face

In [26]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")

In [27]:
from huggingface_hub import login

login(token=HF_TOKEN)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [28]:
import json
import os

folder_path = "./datasets"

list_conversations = []

for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            print(f"Loaded {filename} with {len(data)} conversations.")
            if isinstance(data, list):
                list_conversations.extend(data)

print(f"Total conversations loaded: {len(list_conversations)}")

Loaded 0-preprocessing.json with 294 conversations.
Loaded 1-prompt_variation.json with 1493 conversations.
Loaded 2-multi_turn_conversations.json with 578 conversations.
Loaded 3-cross_faq_conversations.json with 720 conversations.
Total conversations loaded: 3085


In [29]:
list_conversations[:3]

[{'hotel_name': 'Renaissance Riverside Saigon Hotel',
  'conversations': [{'from': 'human',
    'value': 'Bạn có thể cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel được không?'},
   {'from': 'gpt',
    'value': 'Renaissance Riverside Saigon Hotel là khách sạn 5 sao tọa lạc ngay trung tâm Thành phố Hồ Chí Minh. Khách sạn có 336 phòng với cửa sổ nhìn ra thành phố, nhiều tiện nghi hiện đại cùng 2 nhà hàng và bar sang trọng. Vị trí thuận tiện cách sân bay Tân Sơn Nhất khoảng 30 phút lái xe và gần các điểm tham quan nổi tiếng như Nhà hát Lớn, Chợ Bến Thành.'}]},
 {'hotel_name': 'Renaissance Riverside Saigon Hotel',
  'conversations': [{'from': 'human',
    'value': 'Khách sạn Renaissance Riverside Saigon nằm ở địa chỉ nào vậy?'},
   {'from': 'gpt',
    'value': 'Renaissance Riverside Saigon Hotel tọa lạc tại địa chỉ 8 - 15, Ton Duc Thang, Quận 1, Thành phố Hồ Chí Minh, Việt Nam.'}]},
 {'hotel_name': 'Renaissance Riverside Saigon Hotel',
  'conversations': [{'from': 'huma

In [30]:
from datasets import Dataset

cleaned_data = []
for item in list_conversations:
    cleaned_conversations = []
    for conv in item['conversations']:
        cleaned_conversations.append({
            'from': conv.get('from'),
            'value': conv.get('value')
        })
    cleaned_data.append({
        'hotel_name': item.get('hotel_name'),
        'conversations': cleaned_conversations
    })

dataset = Dataset.from_list(cleaned_data)

In [31]:
dataset[:3]

{'hotel_name': ['Renaissance Riverside Saigon Hotel',
  'Renaissance Riverside Saigon Hotel',
  'Renaissance Riverside Saigon Hotel'],
 'conversations': [[{'from': 'human',
    'value': 'Bạn có thể cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel được không?'},
   {'from': 'gpt',
    'value': 'Renaissance Riverside Saigon Hotel là khách sạn 5 sao tọa lạc ngay trung tâm Thành phố Hồ Chí Minh. Khách sạn có 336 phòng với cửa sổ nhìn ra thành phố, nhiều tiện nghi hiện đại cùng 2 nhà hàng và bar sang trọng. Vị trí thuận tiện cách sân bay Tân Sơn Nhất khoảng 30 phút lái xe và gần các điểm tham quan nổi tiếng như Nhà hát Lớn, Chợ Bến Thành.'}],
  [{'from': 'human',
    'value': 'Khách sạn Renaissance Riverside Saigon nằm ở địa chỉ nào vậy?'},
   {'from': 'gpt',
    'value': 'Renaissance Riverside Saigon Hotel tọa lạc tại địa chỉ 8 - 15, Ton Duc Thang, Quận 1, Thành phố Hồ Chí Minh, Việt Nam.'}],
  [{'from': 'human',
    'value': 'Bạn có thể cho tôi biết các hạng phòng hiện c

In [32]:
from huggingface_hub import HfApi

dataset_name = "tyanfarm/hotel-faq-conversations"
api = HfApi()
api.create_repo(repo_id=dataset_name, repo_type="dataset", exist_ok=True)

RepoUrl('https://huggingface.co/datasets/tyanfarm/hotel-faq-conversations', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tyanfarm/hotel-faq-conversations')

In [33]:
dataset.push_to_hub(dataset_name)

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.17s/ shards]
d:\Tyan\Quickom\Simple-Fine-Tune-Llama\finetune_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Tyan\.cache\huggingface\hub\datasets--tyanfarm--hotel-faq-conversations. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/tyanfarm/hotel-faq-conversations/commit/2c160027452260ed9c44eff3f3f446f081d7936e', commit_message='Upload dataset', commit_description='', oid='2c160027452260ed9c44eff3f3f446f081d7936e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/tyanfarm/hotel-faq-conversations', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tyanfarm/hotel-faq-conversations'), pr_revision=None, pr_num=None)